In [1]:
import pandas as pd

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
columns = ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)", "class"]
iris_data = pd.read_csv(url, names=columns)

In [3]:
iris_data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
iris_data['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [5]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")
import time

C:\Users\Przemek\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
X, y = iris_data.drop('class', axis =1), iris_data['class']

In [7]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [9]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

## MLP

### First model

In [10]:
# train the model on train set
cls_model = MLPClassifier(
    hidden_layer_sizes=(16, 8),
    activation='relu',
    solver='adam',
    random_state=42,
    max_iter=100,
    batch_size=32)
cls_model.fit(X_train_sc, y_train)

MLPClassifier(batch_size=32, hidden_layer_sizes=(16, 8), max_iter=100,
              random_state=42)

In [11]:
# print prediction results
y_pred = cls_model.predict(X_test_sc)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



### Model after optimization with GridSearchCV

In [13]:
#defining parameter range
parameters = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam' ],
    'hidden_layer_sizes': [(8, 4), (16,8)],
    'alpha': [0.0001],
    'batch_size': [16,32]
    }

In [14]:
# creating a basic model
cls_model = MLPClassifier(random_state=42)

In [15]:
# finding the best hyperparameters with GridSearchCV with time measurement
start_time = time.time()

grid_search = GridSearchCV(cls_model, parameters)
grid_search.fit(X_train_sc, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

end_time = time.time()
execution_time = round(end_time - start_time,4)
print('----------------------------------')
print(f'Execution time: {execution_time}')

Best parameters: {'activation': 'identity', 'alpha': 0.0001, 'batch_size': 16, 'hidden_layer_sizes': (8, 4), 'solver': 'lbfgs'}
Best score: 0.9666666666666668
----------------------------------
Execution time: 86.8359


In [16]:
# print prediction results
grid_predictions_MLP = grid_search.predict(X_test_sc)
print(classification_report(y_test, grid_predictions_MLP))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



## SVM

### First model

In [17]:
# train the model on train set
svm_model = SVC()
svm_model.fit(X_train_sc, y_train)

SVC()

In [18]:
# print prediction results
y_pred = svm_model.predict(X_test_sc)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



### Model after optimization with GridSearchCV

In [19]:
# defining parameter range
parameters ={
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100, 1000],
    'gamma':[1, 0.1, 0.01, 0.001, 0.0001]
    }

In [20]:
# finding the best hyperparameters with GridSearchCV with time measurement
start_time = time.time()

grid_search = GridSearchCV(svm_model, parameters)
grid_search.fit(X_train_sc, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

end_time = time.time()
execution_time = round(end_time - start_time,4)
print('----------------------------------')
print(f'Execution time: {execution_time}')

Best parameters: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Best score: 0.9833333333333332
----------------------------------
Execution time: 1.473


In [21]:
# print prediction results
grid_predictions_SVM = grid_search.predict(X_test_sc)
print(classification_report(y_test, grid_predictions_SVM))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

